# Mod 4 Project - Starter Notebook

This notebook has been provided to you so that you can make use of the following starter code to help with the trickier parts of preprocessing the Zillow dataset. 

The notebook contains a rough outline the general order you'll likely want to take in this project. You'll notice that most of the areas are left blank. This is so that it's more obvious exactly when you should make use of the starter code provided for preprocessing. 

**_NOTE:_** The number of empty cells are not meant to infer how much or how little code should be involved in any given step--we've just provided a few for your convenience. Add, delete, and change things around in this notebook as needed!

## Some Notes Before Starting

This project will be one of the more challenging projects you complete in this program. This is because working with Time Series data is a bit different than working with regular datasets. In order to make this a bit less frustrating and help you understand what you need to do (and when you need to do it), we'll quickly review the dataset formats that you'll encounter in this project. 

## Wide Format vs Long Format

If you take a look at the format of the data in `zillow_data.csv`, you'll notice that the actual Time Series values are stored as separate columns. Here's a sample: 

<img src='https://raw.githubusercontent.com/learn-co-students/dsc-mod-4-project-seattle-ds-102819/master/images/df_head.png'>

You'll notice that the first seven columns look like any other dataset you're used to working with. However, column 8 refers to the median housing sales values for April 1996, column 9 for May 1996, and so on. This This is called **_Wide Format_**, and it makes the dataframe intuitive and easy to read. However, there are problems with this format when it comes to actually learning from the data, because the data only makes sense if you know the name of the column that the data can be found it. Since column names are metadata, our algorithms will miss out on what dates each value is for. This means that before we pass this data to our ARIMA model, we'll need to reshape our dataset to **_Long Format_**. Reshaped into long format, the dataframe above would now look like:

<img src='https://raw.githubusercontent.com/learn-co-students/dsc-mod-4-project-seattle-ds-102819/master/images/melted1.png'>

There are now many more rows in this dataset--one for each unique time and zipcode combination in the data! Once our dataset is in this format, we'll be able to train an ARIMA model on it. The method used to convert from Wide to Long is `pd.melt()`, and it is common to refer to our dataset as 'melted' after the transition to denote that it is in long format. 

## Wide Format vs Long Format

If you take a look at the format of the data in `zillow_data.csv`, you'll notice that the actual Time Series values are stored as separate columns. Here's a sample: 

<img src='https://raw.githubusercontent.com/learn-co-students/dsc-mod-4-project-seattle-ds-102819/master/images/df_head.png'>

You'll notice that the first seven columns look like any other dataset you're used to working with. However, column 8 refers to the median housing sales values for April 1996, column 9 for May 1996, and so on. This This is called **_Wide Format_**, and it makes the dataframe intuitive and easy to read. However, there are problems with this format when it comes to actually learning from the data, because the data only makes sense if you know the name of the column that the data can be found it. Since column names are metadata, our algorithms will miss out on what dates each value is for. This means that before we pass this data to our ARIMA model, we'll need to reshape our dataset to **_Long Format_**. Reshaped into long format, the dataframe above would now look like:

<img src='https://raw.githubusercontent.com/learn-co-students/dsc-mod-4-project-seattle-ds-102819/master/images/melted1.png'>

There are now many more rows in this dataset--one for each unique time and zipcode combination in the data! Once our dataset is in this format, we'll be able to train an ARIMA model on it. The method used to convert from Wide to Long is `pd.melt()`, and it is common to refer to our dataset as 'melted' after the transition to denote that it is in long format. 

## Helper Functions Provided

Melting a dataset can be tricky if you've never done it before, so you'll see that we have provided a sample function, `melt_data()`, to help you with this step below. Also provided is:

* `get_datetimes()`, a function to deal with converting the column values for datetimes as a pandas series of datetime objects
* Some good parameters for matplotlib to help make your visualizations more readable. 

Good luck!

# Step 1: Load the Data/Filtering for Chosen Zipcodes

In [40]:
## Data Handling
import pandas as pd
import numpy as np

## Visualizations
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

## Settings
%matplotlib inline
plt.style.use('seaborn-talk')
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: f'{x:,.2f}')
pd.set_option('max_rows', 100)

In [29]:
source = '../data/zillow_data.csv'
data = pd.read_csv(source)
data.head(5)

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,1996-07,1996-08,1996-09,1996-10,1996-11,1996-12,1997-01,1997-02,1997-03,1997-04,1997-05,1997-06,1997-07,1997-08,1997-09,1997-10,1997-11,1997-12,1998-01,1998-02,1998-03,1998-04,1998-05,1998-06,1998-07,1998-08,1998-09,1998-10,1998-11,1998-12,1999-01,1999-02,1999-03,1999-04,1999-05,1999-06,1999-07,1999-08,1999-09,1999-10,1999-11,1999-12,2000-01,2000-02,2000-03,2000-04,2000-05,2000-06,2000-07,2000-08,2000-09,2000-10,2000-11,2000-12,2001-01,2001-02,2001-03,2001-04,2001-05,2001-06,2001-07,2001-08,2001-09,2001-10,2001-11,2001-12,2002-01,2002-02,2002-03,2002-04,2002-05,2002-06,2002-07,2002-08,2002-09,2002-10,2002-11,2002-12,2003-01,2003-02,2003-03,2003-04,2003-05,2003-06,2003-07,2003-08,2003-09,2003-10,2003-11,2003-12,2004-01,2004-02,2004-03,2004-04,2004-05,2004-06,2004-07,2004-08,2004-09,2004-10,2004-11,2004-12,2005-01,2005-02,2005-03,2005-04,2005-05,2005-06,2005-07,2005-08,2005-09,2005-10,2005-11,2005-12,2006-01,2006-02,2006-03,2006-04,2006-05,2006-06,2006-07,2006-08,2006-09,2006-10,2006-11,2006-12,2007-01,2007-02,2007-03,2007-04,2007-05,2007-06,2007-07,2007-08,2007-09,2007-10,2007-11,2007-12,2008-01,2008-02,2008-03,2008-04,2008-05,2008-06,2008-07,2008-08,2008-09,2008-10,2008-11,2008-12,2009-01,2009-02,2009-03,2009-04,2009-05,2009-06,2009-07,2009-08,2009-09,2009-10,2009-11,2009-12,2010-01,2010-02,2010-03,2010-04,2010-05,2010-06,2010-07,2010-08,2010-09,2010-10,2010-11,2010-12,2011-01,2011-02,2011-03,2011-04,2011-05,2011-06,2011-07,2011-08,2011-09,2011-10,2011-11,2011-12,2012-01,2012-02,2012-03,2012-04,2012-05,2012-06,2012-07,2012-08,2012-09,2012-10,2012-11,2012-12,2013-01,2013-02,2013-03,2013-04,2013-05,2013-06,2013-07,2013-08,2013-09,2013-10,2013-11,2013-12,2014-01,2014-02,2014-03,2014-04,2014-05,2014-06,2014-07,2014-08,2014-09,2014-10,2014-11,2014-12,2015-01,2015-02,2015-03,2015-04,2015-05,2015-06,2015-07,2015-08,2015-09,2015-10,2015-11,2015-12,2016-01,2016-02,2016-03,2016-04,2016-05,2016-06,2016-07,2016-08,2016-09,2016-10,2016-11,2016-12,2017-01,2017-02,2017-03,2017-04,2017-05,2017-06,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04
0,84654,60657,Chicago,IL,Chicago,Cook,1,"334,200.00","335,400.00","336,500.00","337,600.00","338,500.00","339,500.00","340,400.00","341,300.00","342,600.00","344,400.00","345,700.00","346,700.00","347,800.00","349,000.00","350,400.00","352,000.00","353,900.00","356,200.00","358,800.00","361,800.00","365,700.00","370,200.00","374,700.00","378,900.00","383,500.00","388,300.00","393,300.00","398,500.00","403,800.00","409,100.00","414,600.00","420,100.00","426,200.00","432,600.00","438,600.00","444,200.00","450,000.00","455,900.00","462,100.00","468,500.00","475,300.00","482,500.00","490,200.00","498,200.00","507,200.00","516,800.00","526,300.00","535,300.00","544,500.00","553,500.00","562,400.00","571,200.00","579,800.00","588,100.00","596,300.00","604,200.00","612,200.00","620,200.00","627,700.00","634,500.00","641,000.00","647,000.00","652,700.00","658,100.00","663,300.00","668,400.00","673,400.00","678,300.00","683,200.00","688,300.00","693,300.00","698,000.00","702,400.00","706,400.00","710,200.00","714,000.00","717,800.00","721,700.00","725,700.00","729,900.00","733,400.00","735,600.00","737,200.00","739,000.00","740,900.00","742,700.00","744,400.00","746,000.00","747,200.00","748,000.00","749,000.00","750,200.00","752,300.00","755,300.00","759,200.00","764,000.00","769,600.00","775,600.00","781,900.00","787,900.00","793,200.00","798,200.00","803,100.00","807,900.00","812,900.00","818,100.00","823,100.00","828,300.00","834,100.00","839,800.00","845,600.00","851,800.00","858,000.00","864,400.00","870,700.00","876,200.00","880,700.00","884,400.00","887,600.00","890,500.00","893,300.00","895,500.00","897,300.00","899,000.00","900,400.00","902,000.00","904,400.00","907,100.00","909,700.00","911,900.00","913,000.00","913,000.00","912,000.00","909,300.00","905,300.00","901,400.00","897,900.00","

In [30]:
## Determining number of unique dates in original data

num_dates = len(data.drop(columns = ['RegionName', 'RegionID', 'SizeRank',
                                     'City', 'State', 'Metro',
                                     'CountyName']).copy().columns)
num_dates

265

# Step 4: Reshape from Wide to Long Format

In [31]:
def melt_data(df):
    """
    Takes the zillow_data dataset in wide form or a subset of the zillow_dataset.  
    Returns a long-form datetime dataframe 
    with the datetime column names as the index and the values as the 'values' column.
    
    If more than one row is passes in the wide-form dataset, the values column
    will be the mean of the values from the datetime columns in all of the rows.
    """
    
    melted = pd.melt(df, id_vars=['RegionName', 'RegionID', 'SizeRank', 'City', 'State', 'Metro', 'CountyName'], var_name='time')
    melted['time'] = pd.to_datetime(melted['time'], infer_datetime_format=True)
    melted = melted.dropna(subset=['value'])
    return melted#.groupby('time').aggregate({'value':'mean'})

In [32]:
## Turning date columns into single col w/ dates as values

df_melted = melt_data(data)
df_melted

,RegionName,RegionID,SizeRank,City,State,Metro,CountyName,time,value
0,60657,84654,1,Chicago,IL,Chicago,Cook,1996-04-01,"334,200.00"
1,75070,90668,2,McKinney,TX,Dallas-Fort Worth,Collin,1996-04-01,"235,700.00"
2,77494,91982,3,Katy,TX,Houston,Harris,1996-04-01,"210,400.00"
3,60614,84616,4,Chicago,IL,Chicago,Cook,1996-04-01,"498,100.00"
4,79936,93144,5,El Paso,TX,El Paso,El Paso,1996-04-01,"77,300.00"
...,...,...,...,...,...,...,...,...,...
3901590,1338,58333,14719,Ashfield,MA,Greenfield Town,Franklin,2018-04-01,"209,300.00"
3901591,3293,59107,14720,Woodstock,NH,Claremont,Grafton,2018-04-01,"225,800.00"
3901592,40404,75672,14721,Berea,KY,Richmond,Madison,2018-04-01,"133,400.00"
3901593,81225,93733,14722,Mount Crested Butte,CO,NaN,Gunnison,2018-04-01,"664,400.00"


In [33]:
## Confirming correct number of dates - none missing

num_dates == df_melted.loc[:,'time'].nunique()

True

In [34]:
## Setting index to the new "time" column for processing

df_melted.set_index('time', inplace=True)
df_melted

,RegionName,RegionID,SizeRank,City,State,Metro,CountyName,value
time,,,,,,,,
1996-04-01,60657,84654,1,Chicago,IL,Chicago,Cook,"334,200.00"
1996-04-01,75070,90668,2,McKinney,TX,Dallas-Fort Worth,Collin,"235,700.00"
1996-04-01,77494,91982,3,Katy,TX,Houston,Harris,"210,400.00"
1996-04-01,60614,84616,4,Chicago,IL,Chicago,Cook,"498,100.00"
1996-04-01,79936,93144,5,El Paso,TX,El Paso,El Paso,"77,300.00"
...,...,...,...,...,...,...,...,...
2018-04-01,1338,58333,14719,Ashfield,MA,Greenfield Town,Franklin,"209,300.00"
2018-04-01,3293,59107,14720,Woodstock,NH,Claremont,Grafton,"225,800.00"
2018-04-01,40404,75672,14721,Berea,KY,Richmond,Madison,"133,400.00"


In [38]:
## Confirming index is correct - 265 unique yr/mo combos

num_dates == df_melted.index.nunique()

True

# Step 3: EDA and Visualization

In [41]:
font = {'family' : 'normal',
        'weight' : 'bold',
        'size'   : 22}

mpl.rc('font', **font)

# NOTE: if you visualizations are too cluttered to read, try calling 'plt.gcf().autofmt_xdate()'!

In [61]:
## Checking top 15 states by number of sales
top_15 = df_melted['State'].value_counts()[:15]
top_15

CA    318744
NY    265015
TX    255096
PA    215451
FL    205265
OH    146313
IL    140587
NJ    133030
MI    124765
MA    110112
NC    110024
IN    107300
TN    106349
VA    103160
MN     94384
Name: State, dtype: int64

In [71]:
## Counting number of zip codes in Virginia - first state over 100k entries
df_melted[df_melted['State'] == 'VA']['RegionID'].nunique()

401

In [75]:
df_melted[df_melted['State'] == 'VA']

,RegionName,RegionID,SizeRank,City,State,Metro,CountyName,value
time,,,,,,,,
1996-04-01,23464,67732,63,Virginia Beach,VA,Virginia Beach,Virginia Beach City,"104,400.00"
1996-04-01,23462,67730,90,Virginia Beach,VA,Virginia Beach,Virginia Beach City,"83,400.00"
1996-04-01,22191,67180,194,Woodbridge,VA,Washington,Prince William,"122,600.00"
1996-04-01,23320,67655,203,Chesapeake,VA,Virginia Beach,Chesapeake City,"114,000.00"
1996-04-01,23454,67722,236,Virginia Beach,VA,Virginia Beach,Virginia Beach City,"117,100.00"
...,...,...,...,...,...,...,...,...
2018-04-01,23433,67707,14502,Suffolk,VA,Virginia Beach,Suffolk City,"325,700.00"
2018-04-01,24325,68098,14539,Dugspur,VA,NaN,Carroll,"81,100.00"
2018-04-01,24530,68192,14576,Callands,VA,Danville,Pittsylvania,"56,100.00"


In [91]:
df_melted[df_melted['State'] == "VA"]

,RegionName,RegionID,SizeRank,City,State,Metro,CountyName,value
time,,,,,,,,
1996-04-01,23464,67732,63,Virginia Beach,VA,Virginia Beach,Virginia Beach City,"104,400.00"
1996-04-01,23462,67730,90,Virginia Beach,VA,Virginia Beach,Virginia Beach City,"83,400.00"
1996-04-01,22191,67180,194,Woodbridge,VA,Washington,Prince William,"122,600.00"
1996-04-01,23320,67655,203,Chesapeake,VA,Virginia Beach,Chesapeake City,"114,000.00"
1996-04-01,23454,67722,236,Virginia Beach,VA,Virginia Beach,Virginia Beach City,"117,100.00"
...,...,...,...,...,...,...,...,...
2018-04-01,23433,67707,14502,Suffolk,VA,Virginia Beach,Suffolk City,"325,700.00"
2018-04-01,24325,68098,14539,Dugspur,VA,NaN,Carroll,"81,100.00"
2018-04-01,24530,68192,14576,Callands,VA,Danville,Pittsylvania,"56,100.00"


for each RegionName, I need the

# Step 5: ARIMA Modeling

# Step 6: Interpreting Results

# UNUSED -  Step 2: Data Preprocessing

In [ ]:
# def get_datetimes(df):
#     """
#     Takes a dataframe:
#     returns only those column names that can be converted into datetime objects 
#     as datetime objects.
#     NOTE number of returned columns may not match total number of columns in passed dataframe
#     """
    
#     return pd.to_datetime(df.columns.values, format='%Y-%m')

In [ ]:
# date_cols = data.drop(columns=['RegionID', 'RegionName', 'City', 'State','Metro','CountyName', 'SizeRank'])
# date_cols

In [ ]:
# non_date_cols = ['RegionID', 'RegionName', 'City', 'State','Metro','CountyName', 'SizeRank']
# non_date_cols

In [ ]:
# dt_idx = get_datetimes(data.drop(columns=non_date_cols).copy())
# dt_idx

In [ ]:
# data.drop(columns=non_date_cols).copy().columns = dt_idx

In [ ]:
## Attempting to replace the names of each column w/ updated dates

# cond = data.drop(columns=['RegionID', 'RegionName', 'City', 'State', 'Metro',
#                           'CountyName', 'SizeRank']).copy().columns.values

# choice = dt_idx

# for col in date_col:
#     for date in dt_idx:
#         data.rename(columns = {col:date})
        
# data